### Infêrência Estatística
# Título: Avaliação do impacto da expressões multipalavras nos modelos de linguagem

Este script é similar ao caso base. Mas antes de criar o modelo de linguagem, identificamos as possíveis expressões multipalavras e substituímos a expressão por um único token.

In [1]:
import markovify
import os
import time
import json
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import MWETokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
import math
import numpy as np

Baseamos o código no Read me da biblioteca "Markovify" em: https://github.com/jsvine/markovify/blob/master/README.md

Primeiro criamos uma função para preprocessar o texto.

In [2]:
# Primeiro, criamos uma função para preprocessar os textos
def preprocess(text):
    # Decodificar usando ISO 8859-1 
    text = text.decode('ISO 8859-1')
    # Colocar todas as letras em minúscula
    text = text.lower()
    # Separar as sentenças pelos pontos de finais
    text = text.split('.')
    new_text = []
    for sent in text:
        # Eliminamos as sentenças com menos de 100 caracteres
        if len(sent)> 100:
            # Excluir os espaços no início e fim das sentenças
            new_text.append(sent.strip())
    return(new_text)

Unir os textos em um corpus único para identificar a frequência das palavras e dos bigrams

In [3]:
# Contador de tempo
start_time = time.time()

freq_words = {}
freq_bigram = {}

for (dirpath, _, filenames) in os.walk("Originais/treino"):
    for filename in filenames:
        # Dicionário para armazenar a frequência da palavras

        
        # Ler o arquivo e recuperar a frequência das palavras e dos bibrams
        with open(os.path.join(dirpath, filename), 'rb') as f:
            # Preprocessar o texto lido
            text = preprocess(f.read())
            # Iterando por cada sentença
            for sent in text:
                # tokenizando as sentenças
                sent = nltk.word_tokenize(sent)
                # Recuperando a frequência das palavras da sentença
                freq = FreqDist(sent)
                # Atualizando o dicionário com frequência das palavras
                for i in freq:
                    try:
                        freq_words[i] = freq_words[i] + freq[i]
                    except:
                        freq_words[i] = freq[i]
                # Recuperando os bigram
                bigram = nltk.bigrams(sent)
                for i in bigram:
                    try:
                        freq_bigram[i] = freq_bigram[i] + 1
                    except:
                        freq_bigram[i] = 1      
                        
# Fim do contador de tempo
print ("--- %s seconds ---" % (time.time() - start_time))

--- 50.35356640815735 seconds ---


Com a informação das frequências das palavras e dos bigram, é possível calcular a informação mútua.

In [4]:
# Calculando a Informação Mútua para cada bigram
bigram_MI = {}
n_bigram = len(freq_bigram)
n_word = len(freq_words)
for bi in freq_bigram:
    bigram_MI[bi] = math.log((freq_bigram[bi]/n_bigram)/
                             ((freq_words[bi[0]]/n_bigram)*(freq_words[bi[1]]/n_bigram)))

In [5]:
# Imprimindo o número total de bigrams encontrado
print (len(bigram_MI))

722623


Vamos considerar que um bigram é uma MWE aqueles pares de palavras cuja Informação Mútua ultrapassar um determinado limite.

In [6]:
# Criar uma lista de MWE com os bigrams que ultrapassem o limite assinalado na variável "MI".
mwe = {}
MI = 10.0
for bi in bigram_MI:
    if bigram_MI[bi] > MI:
        mwe[bi] = bigram_MI[bi]
print(len(mwe))
for e in mwe:
    print(bigram_MI[e], e[0], e[1])

12537
11.881205014259596  r e c ô n c a v o   b a i a n o 
10.494910653139705  d i g l i c i d i l   é t e r 
10.158438416518491  c o l a g e m   a u t o m o t i v a 
10.494910653139705  g l i c i d i l   é t e r 
12.79749574613375  p o l i a m i n a   c i c l o a l i f á t i c a 
13.490642926693695  p ' -   d i h i d r o x i d i f e n i l d i m e t i l m e t a n o 
10.446120488970273  a m i n a s   c i c l o a l i f á t i c a s 
10.31258909634575  e t i l e n o   d i a m i n a 
10.446120488970273  e s t e q u i o m é t r i c a   r e s i n a / e n d u r e c e d o r 
12.392030638025586  s c i f i d e r   s c h o l a r 
10.089445545031541  s n   
10.02490702389397  c r u z a m e n t o   i n t e r s i s t e m a 
10.232546388672214  s o f r a m   r e v o l u ç õ e s 
11.293418349357477  m o n o e t i l e n o   g l i c o l 
10.271767101825494  q u i n t e l l a   0 3 a 
11.881205014259596  c o h e r e n t   i n o v a 6 0 
10.82341472011174  b e a m   s p l i t t e r 
10.140738839419091  ó 

11.293418349357477  t u r n   a f f e c t s 
11.005736276905695  c y l i n d e r   b o d y 
10.08944554503154  e x i t   l i n e 
10.600271168797532  r e l e a s e   o p e n i n g 
13.490642926693695  n o n - u n i f o r m   n o n - s t r u c t u r e d 
10.39960047333538  n o n - s t r u c t u r e d   g r i d 
11.092747653895325  p a c k a g e   m e s h i n g 
10.39960047333538  g r i d   r e f i n e m e n t 
11.41120138501386  r e f i n e m e n t   n e a r 
11.293418349357477  p r i s m a t i c   e l e m e n t s 
10.600271168797532  e l e m e n t s   c o n s t r u c t e d 
10.39960047333538  g r i d   s e n s i t i v i t y 
11.69888345746564  s e n s i t i v i t y   t e s t s 
12.79749574613375  m i l l i o n   n o d e s 
10.08944554503154  c o n s e r v a t i o n   l a w s 
11.41120138501386  f r a c t i o n   5øzü5ø^ü
11.005736276905695  t u r b u l e n t   d i f f u s i v i t y 
10.718054204453914  5øxü5øß  f o r m u l a t i o n 
10.600271168797532  a c c u r a c y   b o u n d a r

11.69888345746564  i n f i l t r a d a s   o b l i t e r a r a m 
12.79749574613375  f o t o m i c   r o g r a f i a 
13.490642926693695  c u t í c   u l a s 
10.782592725591485  q u a r t z o s   f r a t u r a d o s 
12.104348565573805  e m p a c o t   a m e n t o 
13.490642926693695  p o n t   u a i s 
11.18805783369965  h á b i t o   b r o t o i d a l 
12.79749574613375  c a l c   í t i c a 
11.092747653895325  p o n t a s   c o n s p í c u a s 
11.005736276905695  p s e u d o m a t r i z   e s v e r d e a d a 
13.490642926693695  f o t   o m i c r o g r a f i a 
12.79749574613375  f e l d s   p a t o s 
10.31258909634575  a u t i g ê n i c a   d e t r í t i c a 
10.05665572220855  t o n s   a m a r r o n z a d o s 
12.392030638025586  a z   u l 
10.089445545031541  f o r m a s s e m   l a m e l a s 
10.232546388672214  e v i d e n c i a r   e s t r u t u r a s / t e x t u r a s 
10.546203947527255  e s t r u t u r a s / t e x t u r a s   m a c r o 
13.490642926693695  p o r c   e n

12.104348565573805  s t e f a n   k a c z m a r z 
13.490642926693695 a n g e n t i h r t e   a u f l o s u n g 
13.490642926693695  s y s t e m e n   l i n e a r e r 
13.490642926693695  l i n e a r e r   g l e i c h u n e n 
11.139267669530218  m u l t i p l i c a t i v e   a l g e b r a i c 
11.005736276905695  a d j e t i v o   s i m u l t a n e o u s 
10.494910653139705  a l g é b r i c a   m u l t i p l i c a t i v a 
10.31258909634575  s i m u l t a n e o u s   m u l t i p l i c a t i v e 
12.392030638025586  s o l o m o n   k u l l b a c k 
11.18805783369965  r i c h a r d   l e i b l e r 
12.104348565573805  h i r o t u g u   a k a i k e 
10.446120488970273  p r o p o s i t a d a m e n t e   e x e m p l i f i c a d o 
10.494910653139705  s e g m e n t a ç ã o   s e m i a u t o m á t i c a 
10.02490702389397  s u p e r i o r m e n t e   d e l i m i t a d a 
10.718054204453914  e x e c u ç õ e s   t e n t a r ã o 
11.005736276905695 t e r r i t o r i a l   p r u d e n t e 
10.08

12.79749574613375  h y d r o t a l c i t e - l i k e   f i l m s 
11.18805783369965  c a l c i n a t i o n   a t m o s p h e r e 
11.092747653895325  p o r o u s   a n o d i c 
13.490642926693695  n i o / z r o 2   c a l c i n e d 
12.104348565573805  n a n o s t r u c t u r e d   n i / c e o 2 
10.158438416518491  t h e r m a l   s i n t e r i n g 
12.79749574613375  m i c r o s t r u c t u r e   c o m p a t i b l e 
10.39960047333538  c o m p a t i b l e   p o r o u s 
11.544732777638382  c a r b o x y l i c   a c i d s 
10.600271168797532  e l e c t r o p h o r e t i c   d e p o s i t i o n 
12.104348565573805  t r a c   t r e n d s 
13.490642926693695  t u n g s t o s i l i c i c   a c i d - s u p p o r t e d 
11.18805783369965  s c a l e - u p   c a p a b i l i t y 
10.355148710764546  s u p p o r t e d   n i - c a t a l y s t s 
11.69888345746564  m a s s - t r a n s f e r   e n h a n c e m e n t 
13.490642926693695  s t a t i c   m i x e r s 
12.79749574613375  h e t e r o g e n

...

13.490642926693695  m i s c a n t h u s   s i n e n s i s 
10.08944554503154  a m i n o á c i d o   f e n i l a l a n i n a 
11.41120138501386  f r o u x a m e n t e   a d e r i d a 
10.851585597078437  g l i c o s í d e o   h i d r o l a s e s 
11.69888345746564  c a r b o h y d r a t e   a c t i v e 
10.31258909634575  a c t i v e   e n z y m e s 
10.494910653139705  e n d o g l i c a n a s e s   h i d r o l i s a m 
10.194806060689366  ²- g l i c o s i d a s e s   f i n a l i z a m 
10.718054204453914  m o n o o x i g e n a s e s   l í t i c a s 
10.31258909634575  a c e s s ó r i o s   a a 9 
11.005736276905695  l p m o s   f ú n g i c a s 
10.925693569232159  p e c t i n a   n ã o - e s t e r i f i c a d a 
10.782592725591485  g l i c o s í d i c a s   ±- 
11.881205014259596  p o l i m e t i l g a l a c t u r o n a t o   l i a s e s 
11.881205014259596  p o l i g a l a c t u r o n a t o   l i a s e s 
12.104348565573805  p e c t i n a / á c i d o   p é c t i c o 
11.544732777638

12.79749574613375  b e e   a p i s 
12.79749574613375  a p i s   m e l l i f e r a 
10.600271168797532  f l u o r e s c e n t   s t a i n i n g 
11.18805783369965  a n t i m i c r o t u b u l e   h e r b i c i d e 
10.782592725591487  c h r o m o s o m e   d o u b l i n g 
13.490642926693695  f u n g i c i d e s   s t i m u l a t e 
11.18805783369965  s t i m u l a t e   c a 2 + 
11.18805783369965  c a 2 +   e f f l u x 
13.490642926693695  c a t i o n i c   p o r p h y r i n 
12.79749574613375  i n t e r m e d i a t e   f i l a m e n t s 
10.232546388672214  s o m e   a n s w e r s 
10.355148710764546  m o r e   q u e s t i o n s 
11.69888345746564  s u p r a v i t a l   s t a i n i n g 
13.490642926693695  h o n e y b e e   s p e r m a t o z o a 
11.41120138501386  i n t r a u t e r i n e   g r o w t h 
11.41120138501386  g r o w t h   r e t a r d a t i o n 
11.005736276905695  i o w a   c o m m u n i t i e s 
11.41120138501386  h e r b i c i d e s - c o n t a m i n a t e d   d r i n

13.490642926693695  s t u d e n t i z e d   r e s i d u a l s 
10.65742958263748  i n s t i t u t o s   m e t r o l ó g i c o s 
11.41120138501386  c o m m u n i t y   c o n c e r n i n g 
10.851585597078437  c o n c e r n i n g   l i m i t s 
10.546203947527255  a l m o s t   a l l 
10.546203947527255  a l l   l a c k 
13.490642926693695  ' d e t e c t i o n   l i m i t s ' 
10.718054204453914  l e t r a   g r a f a d a 
11.69888345746564  a v r a h a m   l o r b e r 
11.41120138501386  n e t   a n a l y t e 
11.257050705186602  a n a l y t e   s i g n a l 
10.77429392277679  x t   - k 
11.293418349357477  x a x a   y k 
10.851585597078437  ây k * v c l   s u b t r a ç ã o 
10.925693569232159  a n n i h i l a t i o n   s t e p 
10.494910653139705  x a v   c o n t e n h a 
10.782592725591485  t r u e   n e t 
10.494910653139705  á l g e b r a   t e n s o r i a l 
10.782592725591487  h - é s i m a   r e p l i c a t a 
10.31258909634575  s o m a s   q u a d r á t i c a s 
10.78259272559

10.271767101825494  s w e e p s   s e n o i d a i s 
10.194806060689366 h y d r o p h o n e   b a s e d 
11.005736276905695  p r i n t e d   p o l y p r o p y l e n e 
11.005736276905695  e l e c t r o n i c s   l e t t e r s 
11.293418349357477  p e r m i t i r ã o   c a t a p u l t a r 
12.104348565573805  v i s i v e l m e n t e   c o n t r a r i a d a 
11.005736276905695  p a u l a   l a b o i s s i è r e 
10.925693569232159  a p r e c i a ç ã o   i n j u s t i f i c a d a 
10.600271168797532  b u s c a r á   d e m o s t r a r 
10.851585597078437  s é r g i o   w u l f f 
11.41120138501386  w u l f f   g o b e t t i 
10.494910653139705  w i t h o u t   d e m o c r a t s 
10.792162176607636  g i l b e r t o   b e r c o v i c i 
11.41120138501386  s i l ê n c i o   e n s u r d e c e d o r 
11.881205014259596  g u s t a v o   b i n e m b o j m 
10.600271168797532  p e r m i t i r ã o   d e s c o r t i n a r 
10.65742958263748  p e t e r   p r e s t o n 
12.104348565573805  e m e r g i

10.271767101825494  s e r r a n o   d e c i d i u 
12.79749574613375  p r e s i d e n t i a l   s u c c e s s i o n 
13.490642926693695 n e w   p o p u l i s m 
13.490642926693695  p o p u l i s m   a r o s e 
11.005736276905695  t r a d i t i o n a l   p o p u l i s m 
11.293418349357477  s t r u c t u r a l   r e f o r m s 
13.490642926693695 n a t i o n a l i s t   b u s i n e s s m e n 
11.41120138501386  p a r t y   b o s s e s 
10.158438416518491  p o p u l i s m   s t i l l 
11.544732777638382  s t i l l   e m p h a s i z e d 
12.392030638025586  t r a d i t i o n a l   a g r a r i a n 
10.546203947527255  v í c i o   r e n t i s t a 
11.005736276905695  r e s p o n s a b i l i z a r   c o n t r a t u a l m e n t e 
11.881205014259596  1 9 7 6 - 1 9 7 9   s t a b i l i z a t i o n 
10.600271168797532  s i t u a c i ó n   a c t u a l 
11.005736276905695  e x p o r t i n g   c o u n t r i e s 
11.005736276905695  e m i n e n t   d o m a i n 
10.194806060689366  p r o p e r t y 

In [7]:
# Função que recebe um sentença e o dicionário com MWE e retorna a sentença com os MWE unidos por "_". 

def uni_MWE(sent, mwe):
    
    mwe_tokenizer = MWETokenizer(list(mwe.keys()))
    sent = nltk.word_tokenize(sent)
    sent = mwe_tokenizer.tokenize(sent)
    sent = TreebankWordDetokenizer().detokenize(sent)
    return (sent)

Para cada arquivos do diretórios especificado lemos e preprocessamos o texto, unimos os bigrams considerados MWE, criamos uma rede de markov com os n-grams (state_size) e combinamos os modelos de todos os arquivos.

In [8]:
# Contador de tempo
start_time = time.time()

# Cadeia de Markov combinada
combined_model = None

# Contagem de arquivos
n_files = 0

# Iterar pelos diretórios e arquivos
for (dirpath, _, filenames) in os.walk("Originais/treino"):
    for filename in filenames:
        n_files += 1
        
        try:
            # Ler o arquivo e criar uma cadeia de Markov para cada arquivo
            with open(os.path.join(dirpath, filename), 'rb') as f:
                
                # Preprocessar o texto lido
                text = preprocess(f.read())
                text_mwe = []
                
                # Unir as MWE
                for sent in text:
                    text_mwe.append(uni_MWE(sent, mwe))
                
                # Treinar o modelos usando as o texto preprocessado e com MWE
                model = markovify.NewlineText(text_mwe, retain_original=False, state_size=2)

                # Combinar modelo criado para um arquivo com a Cadeia de Markov combinada
                if combined_model:
                    combined_model = markovify.combine(models=[combined_model, model])
                else:
                    combined_model = model
        except:
            print("Não foi possível ler o arquivo", filename)
            #pass
                    
        # Contador de arquivos processados
        if n_files % 50 == 0:
            print ("--- %s arquivos processados ---" % n_files, "--- %s seconds ---" % (time.time() - start_time))
    
# Fim do contador de tempo
print ("--- %s seconds ---" % (time.time() - start_time))

--- 50 arquivos processados --- --- 2504.468332529068 seconds ---
--- 100 arquivos processados --- --- 5149.05180311203 seconds ---
--- 7032.4498698711395 seconds ---


In [9]:
# Imprimir um exemplo de sentença aleatória
print(combined_model.make_sentence(max_words=50))

   c o n t u d o ,    q u a n d o   b u s c a   e x í l i o   n o s   e s t a d o s   u n i d o s   m a i s   d e   2 5 %    e   q u e   n ã o   f a z i a m   p a r t e   d a s   o n d a s   i n c i d e n t e s   e   c h o q u e s   r e f l e t i d o s   d a d a   p r e s s õ e s   i n i c i a i s 


In [10]:
# Gravar modelo em JSON em disco
# model_json = combined_model.to_json()
# with open('MarkovChainModel_mwe_MI10.json', 'w') as f:
#     json.dump(model_json, f)

In [11]:
# Carregar modelo em JSON do disco
# with open('MarkovChainModel_mwe_MI10.json', 'r') as f:
#     model_json = markovify.Text.from_json(json.load(f))

Como os textos usados são em português, foi necessário codificar em 'ISO 8859-1', já o modelo JSON está codificado em UTF16. Portanto, ao ler e consultar os termos do modelo é necessário usar a codificação correta.

In [12]:
# Geração de sentença aleatória que iniciam com n-palavras

# Palavras que serãao usadas como semente
palavra_1 = 'a'
palavra_2 = 'petrobras'

# Ajustando a codificação para ser usado no modelo
palavra_1 = ' ' + palavra_1 + ' '
palavra_2 = ' ' + palavra_2
palavra_1 = palavra_1.encode('utf16')[3:]
palavra_2 = palavra_2.encode('utf16')[4:]
seed = palavra_1 + palavra_2

#Criando sentença aleatória iniciando com as palavras sementes
print(combined_model.make_sentence_with_start(seed.decode()))

 a   p e t r o b r a s   p o d e   s o l i c i t a r   u t i l i z a ç ã o   d o   o x i g ê n i o   p o s s u i   d o i s   e l é t r o n s   d e s e m p a r e l h a d o s ,    c a d a   a l t e r n a t i v a   d e   u t i l i z a ç ã o   d o s   r e c u r s o s   n ã o - r e n o v á v e i s 


Do modelo criado pela biblioteca Markovify, extraímos a cadeia de Markov e gravamos em um dicionário

In [13]:
# Extrair cadeia de Markov do modelo
chain = combined_model.to_dict()['chain']
# Transformando a cadeia em um dicionário 
dic = {}
for state in json.loads(chain):
    dic[(state[0][0],state[0][1])] = state[1]

Criamos uma função para consultar a cadeia de Markov. Dado duas palavras, extraímos as próximas palavras possíveis, a contagem delas no conjunto de treinamento e a soma total. Dessa forma conseguimos calcular  $p(x_3|x_2,x_1)$.

In [14]:
# Função que dado duas palavras, retorna um dicionários com as probabilidades das palavras seguintes
def query_chain(palavra_1, palavra_2):
    # Selecionar o dicionário cuja entrada é o par de palavras "palavra_1" e "palavra_2"
    d = dic[(palavra_1, palavra_2)]
    new_d = {}
    # Transformar a contagem de palavras em probabilidade
    for w in d:
        new_d[w] = d[w]/sum(d.values())
    return (new_d)

O calculamos a perplexidade para cada sentença. Dada as primeiras palavras, obtivemos do modelo a probabilidade da palavra seguinte. Em seguida, avançamos pela sentença recuperando as probablidades.

In [15]:
# Dada uma sentença, a função retorna uma lista com as entropias calculada para cada trio de palavras
def entropy(sent):
    # Primeiro unimos os MWE
    sent = uni_MWE(sent, mwe)
    # Quebra a sentença em tokens
    words = nltk.word_tokenize(sent)
    # Variável para guardar a entropia de cada conjunto de palavras
    ln_prob_Xi = []
    # Janela de iteração pela sentença
    for w in range(len(words)-2):
        # Cálculo a entropia
        try:
            ln_prob_Xi.append(- math.log(query_chain(words[w], words[w+1])[words[w+2]]))
        except:
            pass

    return(np.mean(ln_prob_Xi))

In [16]:
# Função que recebe uma string com o caminho e nome da pasta com os arquivos e retorna a perplexidade

def perplexity(dirpath):

    # Início do contador de tempo
    start_time = time.time()
    
    # Variável para armazenar as entropias
    ent = []

    # Contagem de arquivos
    n_files = 0
    # Iterando pelos arquivos da pasta indicada em
    for (dirpath, _, filenames) in os.walk(dirpath):
        
        for filename in filenames:
            n_files += 1
            try:
                # Ler o arquivo e calcular a entropia
                with open(os.path.join(dirpath, filename), 'rb') as f:
                    # Preprocessar o texto lido
                    text = preprocess(f.read())
                    #Calcular a entropia da sentença
                    for sent in text:
                        ent.append(entropy(sent))

            except:
                print("Não foi possível ler o arquivo", filename)
        
                
            # Contador de arquivos processados
            if n_files % 5 == 0:
                print ("--- %s arquivos processados ---" % n_files, "--- %s seconds ---" % (time.time() - start_time))
    
    
    # Transformando a lista de entropias em uma np.array e excluir os valores "nan"
    ent = np.array(ent)
    ent = ent[~np.isnan(ent)]
    
    # Fim do contador de tempo
    print ("--- %s seconds ---" % (time.time() - start_time))
    
    return (math.exp(np.mean(ent)))

In [17]:
perplexity("Originais/teste")

C:\Users\upe2\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\upe2\AppData\Roaming\Python\Python36\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--- 5 arquivos processados --- --- 313.27957248687744 seconds ---
--- 10 arquivos processados --- --- 638.7546446323395 seconds ---
--- 891.452290058136 seconds ---


17.968289148601592